# Scrape stat data

References:
* Main source of scraped statistics: http://japan-mk.blog.jp/
* Additional source of statistics: https://www.mariowiki.com/Mario_Kart_8_Deluxe_in-game_statistics

## Initial setup

### Import libraries

In [1]:
# Async-related imports
import asyncio
import nest_asyncio

# Web scraping imports
import urllib.request
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# Data processing imports
import os
from pathlib import Path
import json
import math
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

### Pre-initialization

In [2]:
# Utility functions
def read_json(filename):
    with open(filename) as fp:
        return json.load(fp)
    
# Pre-initialization
IS_RUNNING_NOTEBOOK = '__file__' not in globals()
IS_RUNNING_BOKEH = __name__.startswith('bokeh')

MAIN_DIRECTORY = os.path.abspath('') if IS_RUNNING_NOTEBOOK else os.path.dirname(__file__)
ROOT_DIRECTORY = str(Path(MAIN_DIRECTORY).parent.absolute())

nest_asyncio.apply()

# Load Japanese-to-English translations
JA_EN_TRANSLATE_TECHNICAL  = read_json(ROOT_DIRECTORY + '/translations/ja_en/technical.json')
JA_EN_TRANSLATE_CHARACTERS = read_json(ROOT_DIRECTORY + '/translations/ja_en/characters.json')
JA_EN_TRANSLATE_FRAMES     = read_json(ROOT_DIRECTORY + '/translations/ja_en/frames.json')
JA_EN_TRANSLATE_TIRES      = read_json(ROOT_DIRECTORY + '/translations/ja_en/tires.json')
JA_EN_TRANSLATE_GLIDERS    = read_json(ROOT_DIRECTORY + '/translations/ja_en/gliders.json')

## Run main logic

### Define low-level logic

In [3]:
async def fetch_page_sources(urls, stat_tab=3):
    page_sources = []
    
    async with async_playwright() as p:
        # Open a new browser
        browser = await p.chromium.launch()
        
        # Open each URL in a new tab to conserve resources
        for (i, url) in enumerate(urls):
            page = await browser.new_page()
            await page.goto(url)

            # Change stat tab
            href = await page.evaluate(f'() => $(".sw_sec{stat_tab}").click()')

            # Grab HTML content
            page_sources.append(await page.content())
            
        # Close the browser session once we're done
        await browser.close()
    
    return page_sources


def extract_table_data_from_src(html_doc, translation_table):
    html_soup = BeautifulSoup(html_doc, 'html.parser')
    main_table = html_soup.select_one('table')

    # Image tags don't have a proper text string to help identify each row, so we're extracting it from the 'alt' attribute
    for img_tag in main_table.select('tbody > tr > td > img'):
        img_tag.replace_with(img_tag['alt'], ' / ')

    # Replace Japanese text with English
    translated_main_table = str(main_table)
    for (ja_text, en_text) in translation_table.items():
        translated_main_table = translated_main_table.replace(ja_text, en_text)

    # Use Pandas to parse the table and standardize the dataframe
    table_df = pd.read_html(translated_main_table, flavor='html5lib')[0]
    table_df = standardize_dataframe(table_df)
    return table_df


def standardize_dataframe(df):
    def _to_numeric(cell):
        try:
            return float(cell)
        except ValueError:
            return cell
    
    def _remove_dupes(arr):
        return list(dict.fromkeys(arr))
    
    # Convert all cells to numeric if possible
    df = df.applymap(_to_numeric)
    
    # Merge the multi-index to a single index
    df = df.set_axis(
        np.apply_along_axis(
            # HACK: We need to wrap the row in an numpy array since a string is an
            # array of characters and the output of the function expects a 1D array of sorts
            # Additionally, not casting it as an 'object' will trim the string contents to the shortest string size
            lambda row: np.array(" - ".join( _remove_dupes(row) ), dtype='object'),
            axis=1,
            arr=np.array(df.columns.tolist(), dtype = 'object')
        ),
        axis='columns'
    )
    
    # Remove rows that have all strings. This works by collecting all present types across each row, getting its
    # string representation, removing the 'str' type and checking if any other types remain
    df = df[
        np.apply_along_axis(
            func1d=lambda row: len(set([type(cell).__name__ for cell in row]) - {'str'}),
            axis=1,
            arr=df.to_numpy()
        ) == 1
    ]
    
    # Remove the ending slashes from the first column
    df.iloc[:, 0] = df.iloc[:, 0].str.rstrip(to_strip=' /')
    
    return df

### Scrape stats from japan-mk.blog.jp

In [4]:
async def scrape_sources():
    return await fetch_page_sources([
        'http://japan-mk.blog.jp/mk8dx.st-c', # Characters
        'http://japan-mk.blog.jp/mk8dx.st-f', # Frames
        'http://japan-mk.blog.jp/mk8dx.st-t', # Tires
        'http://japan-mk.blog.jp/mk8dx.st-g', # Gliders
    ], stat_tab=1)

MAIN_LOOP = asyncio.get_event_loop()

print('Fetching all page statistics sources...', end='')

(
    char_stat_src,
    frame_stat_src,
    tire_stat_src,
    glider_stat_src,
) = MAIN_LOOP.run_until_complete(scrape_sources())

print('done!')

Fetching all page statistics sources...done!


### Extract data from sources

In [5]:
print('Extracting tabular data from sources...', end='')

character_stats_df = extract_table_data_from_src(char_stat_src,   JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_CHARACTERS).set_index('Character')
frame_stats_df     = extract_table_data_from_src(frame_stat_src,  JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_FRAMES).set_index('Parts')
tire_stats_df      = extract_table_data_from_src(tire_stat_src,   JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_TIRES).set_index('Parts')
glider_stats_df    = extract_table_data_from_src(glider_stat_src, JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_GLIDERS).set_index('Parts')

print('done!')

Extracting tabular data from sources...done!


In [6]:
dframes = {
    'Character': character_stats_df,
    'Frame':     frame_stats_df,
    'Tire':      tire_stats_df,
    'Glider':    glider_stats_df,
}

total_combos = math.prod([dframe.shape[0] for dframe in dframes.values()])
print(f'Total possible combos: {total_combos}')

Total possible combos: 26656


### Generate all possible kart combos

NOTE: To whomever is reading this: YES, I know this code isn't using pandas 'good practices' but this is a one-time operation that only takes at most a few seconds. I value readability over performance as this is not my full time job and I'd rather not lose my mind trying to relearn a sub-language (pandas) inside a language (python).

In [7]:
# Given the index for each combo feature, compute the total in-game stats
def compute_combo_stats(character_index, frame_index, tire_index, glider_index):
    character_stats = dframes['Character'].iloc[character_index]
    frame_stats     = dframes['Frame'].iloc[frame_index]
    tire_stats      = dframes['Tire'].iloc[tire_index]
    glider_stats    = dframes['Glider'].iloc[glider_index]

    part_names = (character_stats.name, frame_stats.name, tire_stats.name, glider_stats.name)
    total_stats = character_stats + frame_stats + tire_stats + glider_stats
    
    return part_names, total_stats

In [8]:
all_combos = []
combo_bar = tqdm(range(total_combos))

# Generate all possible kart combos' stats and string them together in a DataFrame
for c_index in range(dframes['Character'].shape[0]):
    for f_index in range(dframes['Frame'].shape[0]):
        for t_index in range(dframes['Tire'].shape[0]):
            for g_index in range(dframes['Glider'].shape[0]):
                combo_bar.update(1)

                part_names, total_stats = compute_combo_stats(c_index, f_index, t_index, g_index)
                total_stats['Character'] = part_names[0]
                total_stats['Frame'] = part_names[1]
                total_stats['Tire'] = part_names[2]
                total_stats['Glider'] = part_names[3]
                all_combos.append(total_stats)

all_combos_df = (
    pd.concat(all_combos, axis=1)
        .transpose()
        .set_index(['Character', 'Frame', 'Tire', 'Glider'])
        .infer_objects()
)

all_combos_df.to_csv(f'{ROOT_DIRECTORY}/stats/All-Combos.csv', index=True)

  0%|          | 0/26656 [00:00<?, ?it/s]

In [9]:
# TIP: Ctrl + Enter/Return to check out random combos
all_combos_df.sample(n=3).head()

,,,,Speed - Land,Speed - Water,Speed - Air,Speed - Anti-G,Acceleration,Weight,Handling - Land,Handling - Water,Handling - Air,Handling - Anti-G,Off-Road Traction,Mini Turbo,Invincibility
Character,Frame,Tire,Glider,,,,,,,,,,,,,
Baby Peach / Baby Daisy,Varmint / City Tripper,Off-Road / Retro Off-Road,Peach Parasol / Parafoil / Bowser Kite / MKTV Parafoil,6.0,8.0,4.0,5.0,15.0,5.0,17.0,16.0,16.0,16.0,15.0,16.0,13.0
Peach / Daisy / Yoshi / Birdo,Prancer,Standard / Blue Standard,Cloud Glider / Parachute / Flower Glider / Paper Glider,12.0,13.0,13.0,12.0,11.0,7.0,12.0,11.0,13.0,11.0,11.0,14.0,9.0
Baby Peach / Baby Daisy,Inkstriker,Standard / Blue Standard,Wario Wing / Plane Glider / Gold Glider / Paraglider,7.0,7.0,10.0,9.0,12.0,7.0,16.0,16.0,16.0,17.0,14.0,15.0,13.0


### Save extracted data to CSV

In [10]:
print('Saving extracted data to CSVs...', end='')

Path(f'{ROOT_DIRECTORY}/stats').mkdir(parents=True, exist_ok=True)
for (name, df) in dframes.items():
    df.to_csv(f'{ROOT_DIRECTORY}/stats/{name}.csv', index=True)
    
print('done!')

Saving extracted data to CSVs...done!
